In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima_model import ARIMA


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler


import pickle

In [2]:
df = pd.read_csv('../data/raw_recording_1.csv')
df

,x_pos,y_pos,channel_1,channel_2,channel_3,channel_4
0,2272.0,659.0,319,19,0,83
1,2268.0,661.0,22,32,0,83
2,2265.0,664.0,22,31,12,83
3,2265.0,664.0,22,31,16,111
4,2265.0,664.0,21,31,16,111
...,...,...,...,...,...,...
122532,1577.0,1148.0,13,28,6,351
122533,1577.0,1148.0,13,28,6,351
122534,1581.0,1148.0,13,28,6,351
122535,1581.0,1148.0,13,28,6,350


In [3]:
# df.drop(columns=['time', 'start', 'end'], inplace=True)

In [4]:
for i in range(1,500):

    df[f'channel_1_{i}'] = df['channel_1'].shift(i)
    df[f'channel_2_{i}'] = df['channel_2'].shift(i)
    df[f'channel_3_{i}'] = df['channel_3'].shift(i)
    df[f'channel_4_{i}'] = df['channel_4'].shift(i)

In [5]:
df['x_diff'] = df['x_pos'].diff()
df['y_diff'] = df['y_pos'].diff()

In [6]:
df.dropna(inplace=True)

In [7]:
X = df.drop(columns=['x_pos', 'y_pos', 'x_diff', 'y_diff'])
y = list(df['x_diff'])

W = X
z = list(df['y_diff'])

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)
W_train, W_test, z_train, z_test = train_test_split(W, z, random_state = 42)

In [ ]:
lr_x = RandomForestRegressor(n_estimators=25)
lr_x.fit(X_train, y_train)

lr_y = RandomForestRegressor(n_estimators=25)
lr_y.fit(W_train, z_train)

In [ ]:
lr_x.score(X_train, y_train), lr_x.score(X_test, y_test)

In [ ]:
lr_y.score(W_train, z_train), lr_y.score(W_test, z_test)

In [ ]:
plt.figure(figsize=(100,100))
plt.plot(y, z);
plt.plot(lr_x.predict(X), lr_y.predict(W));


In [ ]:
plt.figure(figsize=(100,100))
plt.plot(y[:10000]);
plt.plot(lr_x.predict(X)[:10000]);


In [ ]:
plt.figure(figsize=(100,100))
plt.plot(z[:10000]);
plt.plot(lr_y.predict(W)[:10000]);


In [ ]:
with open('../pickled_models/rf_x.pkl', 'wb') as f:
    pickle.dump(lr_x, f)


with open('../pickled_models/rf_y.pkl', 'wb') as f:
    pickle.dump(lr_y, f)
